In [1]:
import sys
import os

# Activate virtual environment
venv_path = os.path.expanduser("~/mujoco_ws/venv")
if os.path.exists(venv_path):
    sys.path.insert(0, os.path.join(venv_path, "lib", "python3.11", "site-packages"))
    print(f"✓ Virtual environment activated: {venv_path}")
    print(f"✓ Python executable: {sys.executable}")
else:
    print(f"⚠ Virtual environment not found at {venv_path}")
    print(f"Currently using Python: {sys.executable}")

✓ Virtual environment activated: /home/cvincen6/mujoco_ws/venv
✓ Python executable: /home/cvincen6/mujoco_ws/venv/bin/python


## Setup and Imports

In [2]:
import numpy as np
import mujoco
import gymnasium as gym
from gymnasium import spaces
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import time
import torch
from datetime import timedelta
import os

print(f"PyTorch using: {'cuda' if torch.cuda.is_available() else 'cpu'} device")
print(f"NumPy version: {np.__version__}")
print(f"MuJoCo version: {mujoco.__version__}")

ModuleNotFoundError: No module named 'numpy'

## Box Pick-and-Place Environment 

In [ ]:
python -m venv ~/mujoco_ws/venv

In [ ]:
python -m venv ~/mujoco_ws/venv

In [ ]:
class PickPlaceEnv(gym.Env):
    """Environment for training Franka Panda robot to pick and place colored boxes into matching buckets."""
    
    def __init__(self, model_path='model/franka_panda/pick_place_scene.xml', render_mode=None):
        super().__init__()
        self.render_mode = render_mode
        self.viewer = None
        
        # Load model
        try:
            self.model = mujoco.MjModel.from_xml_path(model_path)
        except FileNotFoundError:
            raise FileNotFoundError(f"Model not found at {model_path}")
        
        self.data = mujoco.MjData(self.model)
        self.dt = self.model.opt.timestep
        
        # Action space: 7 arm joints + 1 gripper = 8D (Franka has 8 control inputs)
        self.action_space = spaces.Box(
            low=-1.0, high=1.0, shape=(8,), dtype=np.float32
        )
        
        # Observation: 7 arm angles + 7 velocities + 12 box positions (36D) + 1 gripper position = 51D
        self.observation_space = spaces.Box(
            low=-np.inf, high=np.inf, shape=(51,), dtype=np.float32
        )
        
        # Box color mapping
        self.boxes = {
            'red': ['red_box_0', 'red_box_1', 'red_box_2'],
            'blue': ['blue_box_0', 'blue_box_1', 'blue_box_2'],
            'green': ['green_box_0', 'green_box_1', 'green_box_2'],
            'yellow': ['yellow_box_0', 'yellow_box_1', 'yellow_box_2']
        }
        
        # Bucket target centers (adapted for Franka workspace)
        self.bucket_targets = {
            'red': np.array([0.6, 0.6, 0.15]),
            'blue': np.array([-0.6, 0.6, 0.15]),
            'green': np.array([0.6, -0.6, 0.15]),
            'yellow': np.array([-0.6, -0.6, 0.15])
        }
        
        self.max_episode_steps = 2000
        self.step_count = 0
    
    def _get_obs(self):
        """Get current observation."""
        # Arm state (7 joints for Franka)
        q = self.data.qpos[:7].copy()
        qv = self.data.qvel[:7].copy()
        
        # Gripper state (joint index 7 for Franka)
        gripper_pos = self.data.qpos[7] if len(self.data.qpos) > 7 else 0.0
        
        # Box positions
        box_positions = []
        for color in ['red', 'blue', 'green', 'yellow']:
            for i in range(3):
                box_name = f'{color}_box_{i}'
                try:
                    pos = self.data.body(box_name).xpos.copy()
                    box_positions.append(pos)
                except:
                    box_positions.append(np.array([0.0, 0.0, 0.0]))
        
        box_positions = np.array(box_positions).flatten()
        obs = np.concatenate([q, qv, box_positions, [gripper_pos]]).astype(np.float32)
        return obs
    
    def reset(self, seed=None, options=None):
        """Reset the environment."""
        super().reset(seed=seed)
        mujoco.mj_resetData(self.model, self.data)
        self.step_count = 0
        obs = self._get_obs()
        info = {}
        return obs, info
    
    def step(self, action):
        """Execute one step within the environment."""
        # Clamp actions to valid range
        action = np.clip(action, self.action_space.low, self.action_space.high)
        
        # Apply actions directly to all 8 control inputs
        self.data.ctrl[:] = action
        
        # Simulate one step
        mujoco.mj_step(self.model, self.data)
        self.step_count += 1
        
        # Get observation
        obs = self._get_obs()
        
        # Calculate reward
        reward = self._calculate_reward()
        
        # Check if episode is done
        terminated = self.step_count >= self.max_episode_steps
        truncated = False
        
        return obs, reward, terminated, truncated, {}
    
    def _calculate_reward(self):
        """Calculate reward for the current state."""
        reward = 0.0
        
        # Reward for boxes in correct buckets
        for color, target_pos in self.bucket_targets.items():
            for i in range(3):
                box_name = f'{color}_box_{i}'
                try:
                    box_pos = self.data.body(box_name).xpos
                    # Distance to target
                    distance = np.linalg.norm(box_pos - target_pos)
                    
                    # Reward if close to bucket
                    if distance < 0.2:
                        reward += 10.0 * (1.0 - distance / 0.2)
                    
                    # Bonus if inside bucket
                    if distance < 0.15:
                        reward += 50.0
                except:
                    pass
        
        return reward

: 

## Train Pick-and-Place Model

In [ ]:
def train_pick_place(total_timesteps=300000):
    """Train the UR5e robot for pick-and-place with gripper."""
    print("=" * 70)
    print("Training UR5e Robot for Pick-and-Place with Gripper")
    print("=" * 70)
    print()
    print("Task: Pick colored boxes and place them in matching colored buckets")
    print("  - Red boxes → Red bucket at (0.7, 0.7)")
    print("  - Blue boxes → Blue bucket at (-0.7, 0.7)")
    print("  - Green boxes → Green bucket at (0.7, -0.7)")
    print("  - Yellow boxes → Yellow bucket at (-0.7, -0.7)")
    print()
    print(f"Using {('cuda' if torch.cuda.is_available() else 'cpu')} device")
    print()
    
    # Create environment
    env = make_vec_env(
        lambda: PickPlaceEnv(model_path='model/franka_panda/pick_place_scene.xml'),
        n_envs=1
    )
    
    # Create PPO model with more training
    model = PPO(
        "MlpPolicy",
        env,
        learning_rate=3e-4,
        n_steps=2048,
        batch_size=64,
        n_epochs=10,
        gamma=0.99,
        gae_lambda=0.95,
        clip_range=0.2,
        ent_coef=0.01,
        verbose=1,
        tensorboard_log=None
    )
    
    print("Training started...")
    print(f"Total timesteps: {total_timesteps:,}\n")
    
    start_time = time.time()
    model.learn(total_timesteps=total_timesteps)
    elapsed_time = time.time() - start_time
    elapsed_str = str(timedelta(seconds=int(elapsed_time)))
    
    # Save the model
    model.save("franka_pick_place_ppo")
    print("\n" + "=" * 70)
    print(f"Training complete!")
    print(f"Time elapsed: {elapsed_str}")
    print(f"Model saved as 'franka_pick_place_ppo'")
    print("=" * 70)
    
    env.close()
    return model

print("✓ train_pick_place() function defined")

✓ train_pick_place() function defined


In [ ]:
model_pp = train_pick_place()

Training UR5e Robot for Pick-and-Place with Gripper

Task: Pick colored boxes and place them in matching colored buckets
  - Red boxes → Red bucket at (0.7, 0.7)
  - Blue boxes → Blue bucket at (-0.7, 0.7)
  - Green boxes → Green bucket at (0.7, -0.7)
  - Yellow boxes → Yellow bucket at (-0.7, -0.7)

Using cuda device

Using cuda device
Training started...
Total timesteps: 300,000

---------------------------------
| rollout/           |          |
|    ep_len_mean     | 2e+03    |
|    ep_rew_mean     | 0        |
| time/              |          |
|    fps             | 1550     |
|    iterations      | 1        |
|    time_elapsed    | 1        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 2e+03       |
|    ep_rew_mean          | 0           |
| time/                   |             |
|    fps                  | 1208        |
|    iterations        

## Summary

The UR5e robot with gripper has been trained using PPO to:

1. **Perceive the environment**: Observe arm state, gripper position, and all box locations
2. **Learn grasping**: Pick up colored boxes using the gripper
3. **Place precisely**: Put each box into the matching colored bucket (red→red, blue→blue, etc.)
4. **Optimize efficiency**: Complete the sorting task with minimal wasted motion

**Task Configuration**:
- 12 colored boxes (3 red, 3 blue, 3 green, 3 yellow)
- 4 matching colored buckets at corners (±0.7, ±0.7)
- 8-dimensional action space: 6 arm joints + 2 gripper fingers
- 50-dimensional observation space: arm state + box positions + gripper state


## Visualize Pick-and-Place Model

In [ ]:
def visualize_pick_place(num_episodes=1, deterministic=True):
    """Visualize pick-and-place model with real-time rendering."""
    from stable_baselines3 import PPO
    import mujoco.viewer
    
    print("=" * 70)
    print("Visualizing Pick-and-Place Model with Franka Gripper")
    print("=" * 70)
    print(f"Episodes: {num_episodes}")
    print(f"Deterministic: {deterministic}")
    print("\nViewing boxes being picked and placed into matching buckets...")
    print("=" * 70)
    print()
    
    loaded_model = PPO.load("franka_pick_place_ppo")
    
    for ep in range(num_episodes):
        print(f"\n--- Episode {ep + 1} ---")
        
        env = PickPlaceEnv(model_path='model/franka_panda/pick_place_scene.xml')
        obs, _ = env.reset()
        
        episode_reward = 0
        step_count = 0
        terminated = False
        truncated = False
        
        viewer = mujoco.viewer.launch_passive(env.model, env.data)
        
        try:
            while not (terminated or truncated):
                action, _ = loaded_model.predict(obs, deterministic=deterministic)
                obs, reward, terminated, truncated, _ = env.step(action)
                episode_reward += reward
                step_count += 1
                viewer.sync()
        finally:
            viewer.close()
        
        print(f"Steps: {step_count}, Reward: {episode_reward:.2f}")
        env.close()
    
    print("\n" + "=" * 70)
    print("Visualization complete!")
    print("=" * 70)

print("✓ visualize_pick_place() function defined")

✓ visualize_pick_place() function defined


In [ ]:
visualize_pick_place(num_episodes=5)

Visualizing Pick-and-Place Model with Franka Gripper
Episodes: 5
Deterministic: True

Viewing boxes being picked and placed into matching buckets...


--- Episode 1 ---
Steps: 2000, Reward: 0.00

--- Episode 2 ---
Steps: 2000, Reward: 0.00

--- Episode 3 ---
Steps: 2000, Reward: 0.00

--- Episode 4 ---
Steps: 2000, Reward: 0.00

--- Episode 5 ---
Steps: 2000, Reward: 0.00

Visualization complete!


## Visualize the Scene

In [ ]:
# Visualize the empty scene without training
import mujoco.viewer

model_path = 'model/franka_panda/pick_place_scene.xml'
model = mujoco.MjModel.from_xml_path(model_path)
data = mujoco.MjData(model)

# Run simulation for a few seconds to see the scene
with mujoco.viewer.launch_passive(model, data) as viewer:
    # Run for 2 seconds
    start = time.time()
    while viewer.is_running() and time.time() - start < 2.0:
        step_start = data.time
        while (data.time - step_start) < model.opt.timestep * 10:
            mujoco.mj_step(model, data)
        
        viewer.sync()
        time.sleep(0.001)